In [157]:
## Utilities
from __future__ import print_function
import argparse
import random
import time
import os
import logging
from timeit import default_timer as timer

## Libraries
import numpy as np

## Torch
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
import torch.optim as optim

## Custrom Imports
# from src.logger_v1 import setup_logs
# from src.data_reader.dataset import RawDataset, ReverseRawDataset, RawXXreverseDataset
# from src.training_v1 import trainXXreverse, snapshot
# from src.validation_v1 import validationXXreverse
# from src.model.model import CDCK5, CDCK6
############ Control Center and Hyperparameter ###############
run_name = "cdc" + time.strftime("-%Y-%m-%d_%H_%M_%S")
print(run_name)
logger = logging.getLogger("cdc")

cdc-2021-03-21_17_25_54


In [28]:
def rolling_window(a, window):
    shape =  (a.shape[0] - window + 1, window) + a.shape[1:]
    strides = (a.strides[0],) + a.strides
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides, writeable = False)

In [113]:
class ScheduledOptim(object):
    """A simple wrapper class for learning rate scheduling"""

    def __init__(self, optimizer, n_warmup_steps):
        self.optimizer = optimizer
        self.d_model = 128 
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0 
        self.delta = 1

    def state_dict(self):
        self.optimizer.state_dict()

    def step(self):
        """Step by the inner optimizer"""
        self.optimizer.step()

    def zero_grad(self):
        """Zero out the gradients by the inner optimizer"""
        self.optimizer.zero_grad()

    def increase_delta(self):
        self.delta *= 2

    def update_learning_rate(self):
        """Learning rate scheduling per step"""

        self.n_current_steps += self.delta
        new_lr = np.power(self.d_model, -0.5) * np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = new_lr
        return new_lr

In [133]:
class TETrainDataset(data.Dataset):
    def __init__(self, fault=[1,2,3,4,5,6], window=20):
        """ fault: [1,2,3,4,5,6]
            window: 20
        """
        self.window = window 
        temp = torch.from_numpy(rolling_window(np.loadtxt('data/d00.dat').T, window))
        self.sample = [temp]
        self.label = [0 for _ in temp]

        for label in fault:
            if label < 10:
                num = '0' + str(label)
            else:
                num = str(label)
            temp = torch.from_numpy(rolling_window(np.loadtxt('data/d' + num + '.dat'), window))
            self.sample.append(temp)
            self.label.extend([label for _ in temp])
        
        self.sample = torch.cat(self.sample,0).float()
        self.label = torch.tensor(self.label).float()
        
    def __len__(self):
        return len(self.sample)

    def __getitem__(self, index):
        return self.sample[index], self.label[index]

class TETestDataset(data.Dataset):
    def __init__(self, fault=[1,2,3,4,5,6], window=20):
        """ fault: [1,2,3,4,5,6]
            window: 20
        """
        self.window = window 
        temp = torch.from_numpy(rolling_window(np.loadtxt('data/d00_te.dat'), window))
        self.sample = [temp]
        self.label = [0 for _ in temp]

        for label in fault:
            if label < 10:
                num = '0' + str(label)
            else:
                num = str(label)
            temp = torch.from_numpy(rolling_window(np.loadtxt('data/d' + num + '_te.dat'), window))
            self.sample.append(temp)
            if window <= 160:
                self.label.extend([0 for _ in range(160-window+1)])
                self.label.extend([label for _ in range(800)])
            else:
                self.label.extend([label for _ in range(960-window+1)])
        
        self.sample = torch.cat(self.sample,0).float()
        self.label = torch.tensor(self.label).float()
        assert self.sample.shape[0] == self.label.shape[0]
        
    def __len__(self):
        return len(self.sample)

    def __getitem__(self, index):
        return self.sample[index], self.label[index]

In [158]:
class CPC(nn.Module):
    def __init__(self, input_size, timestep, batch_size, seq_len):

        super(CPC, self).__init__()

        self.batch_size = batch_size
        self.seq_len = seq_len
        self.timestep = timestep
        self.encoder = nn.Sequential( # downsampling factor = 160
            nn.Linear(input_size,32, bias=False),
            nn.BatchNorm1d(self.seq_len+self.timestep),
            nn.ReLU(inplace=True),
            nn.Linear(32,32, bias=False),
            nn.BatchNorm1d(self.seq_len+self.timestep),
            nn.ReLU(inplace=True),
        )
        self.gru = nn.GRU(32, 16, num_layers=1, bidirectional=False, batch_first=True)
        self.Wk  = nn.ModuleList([nn.Linear(16, 32) for i in range(timestep)])
        self.softmax  = nn.Softmax()
        self.lsoftmax = nn.LogSoftmax()

        def _weights_init(m):
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # initialize gru
        for layer_p in self.gru._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    nn.init.kaiming_normal_(self.gru.__getattr__(p), mode='fan_out', nonlinearity='relu')

        self.apply(_weights_init)

    def init_hidden(self, batch_size, use_gpu=True):
        if use_gpu: return torch.zeros(1, batch_size, 16).cuda()
        else: return torch.zeros(1, batch_size, 16)

    def forward(self, x, hidden):
        batch = x.size()[0]
        # t_samples = torch.randint(self.seq_len/160-self.timestep, size=(1,)).long() # randomly pick time stamps
        # input sequence is N*C*L, e.g. 8*1*20480
        z = self.encoder(x)
        # encoded sequence is N*C*L, e.g. 8*512*128
        # reshape to N*L*C for GRU, e.g. 8*128*512
        encode_samples = z[:,-1*self.timestep:,:].transpose(0,1)
        # z = z.transpose(1,2)
        nce = 0 # average over timestep and batch
        # encode_samples = torch.empty((self.timestep,batch,512)).float() # e.g. size 12*8*512
        # for i in np.arange(1, self.timestep+1):
        #     encode_samples[i-1] = z[:,t_samples+i,:].view(batch,512) # z_tk e.g. size 8*512
        forward_seq = z[:,:self.seq_len,:] # e.g. size 8*100*512
        output, hidden = self.gru(forward_seq, hidden) # output size e.g. 8*100*256
        c_t = output[:,-1,:].view(batch, -1) # c_t e.g. size 8*256
        pred = torch.empty((self.timestep,batch,32)).float().to(x.device) # e.g. size 12*8*512
        for i in np.arange(0, self.timestep):
            pred[i] = self.Wk[i](c_t) # Wk*c_t e.g. size 8*512
        for i in np.arange(0, self.timestep):
            total = torch.mm(encode_samples[i], torch.transpose(pred[i],0,1)) # e.g. size 8*8
            correct = torch.sum(torch.eq(torch.argmax(self.softmax(total), dim=0).cpu(), torch.arange(0, batch))) # correct is a tensor
            nce += torch.sum(torch.diag(self.lsoftmax(total))) # nce is a tensor
        nce /= -1.*batch*self.timestep
        accuracy = 1.*correct.item()/batch

        return accuracy, nce

    def predict(self, x, hidden):
        batch = x.size()[0]
        # input sequence is N*C*L, e.g. 8*1*20480
        z = self.encoder(x)
        # encoded sequence is N*C*L, e.g. 8*512*128
        # reshape to N*L*C for GRU, e.g. 8*128*512
        z = z.transpose(1,2)
        output, hidden = self.gru(z, hidden) # output size e.g. 8*128*256

        return output, hidden # return every frame
        #return output[:,-1,:], hidden # only return the last frame per utt

In [163]:
def train(args, model, device, train_loader, optimizer, epoch, batch_size):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        seq, label = data
        seq, label = seq.to(device), label.to(device)
        # seq = seq.to(device) # add channel dimension
        optimizer.zero_grad()
        hidden = model.init_hidden(len(seq), use_gpu=True)
        acc, loss = model(seq, hidden)

        loss.backward()
        optimizer.step()
        lr = optimizer.update_learning_rate()
        if batch_idx % args.log_interval == 0:
            logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tlr:{:.5f}\tAccuracy: {:.4f}\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), lr, acc, loss.item()))

def validation(args, model, device, data_loader, batch_size):
    logger.info("Starting Validation")
    model.eval()
    total_loss = 0
    total_acc  = 0 

    with torch.no_grad():
        for data in data_loader:
            seq, label = data
            seq, label = seq.to(device), label.to(device)
            # data = data.float().unsqueeze(1).to(device) # add channel dimension
            hidden = model.init_hidden(len(seq), use_gpu=True)
            acc, loss = model(seq, hidden)
            total_loss += len(seq) * loss 
            total_acc  += len(seq) * acc

    total_loss /= len(data_loader.dataset) # average loss
    total_acc  /= len(data_loader.dataset) # average acc

    logger.info('===> Validation set: Average loss: {:.4f}\tAccuracy: {:.4f}\n'.format(
                total_loss, total_acc))

    return total_acc, total_loss

In [162]:
def main():
    ## Settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--train-raw', default='LibriSpeech/train-Librispeech.h5')
    parser.add_argument('--validation-raw', default='LibriSpeech/validation-Librispeech.h5')
    parser.add_argument('--eval-raw', default='LibriSpeech/eval-Librispeech.h5')
    parser.add_argument('--train-list', default='LibriSpeech/list/train.txt')
    parser.add_argument('--validation-list', default='LibriSpeech/list/validation.txt')
    parser.add_argument('--eval-list')
    parser.add_argument('--logging-dir', default='snapshot/cdc/',
                        help='model save directory')
    parser.add_argument('--epochs', type=int, default=60, metavar='N',
                        help='number of epochs to train')
    parser.add_argument('--n-warmup-steps', type=int, default=50)
    parser.add_argument('--batch-size', type=int, default=10, 
                        help='batch size')
    parser.add_argument('--window', type=int, default=20, 
                        help='window length to sample from each utterance')
    parser.add_argument('--timestep', type=int, default=10) 
    parser.add_argument('--masked-frames', type=int, default=20)
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
#     args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    print('use_cuda is', use_cuda)
    global_timer = timer() # global timer
    logger = setup_logs(args.logging_dir, run_name) # setup logs
    device = torch.device("cuda" if use_cuda else "cpu")
    model = CPC(52, args.timestep, args.batch_size, args.window).to(device)
    #model = CDCK5(args.timestep, args.batch_size, args.audio_window).to(device)
    #model = CDCK6(args.timestep, args.batch_sz
    params = {'num_workers': 0,
              'pin_memory': False} if use_cuda else {}

    logger.info('===> loading train, validation and eval dataset')
    training_set   = TETrainDataset(window=args.window+args.timestep)
    #training_set   = ReverseRawDataset(args.train_raw, args.train_list, args.audio_window)
    #training_set   = RawXXreverseDataset(args.train_raw, args.train_list, args.audio_window)
    test_set = TETestDataset(window=args.window+args.timestep)
    #validation_set = ReverseRawDataset(args.validation_raw, args.validation_list, args.audio_window)
    #validation_set = RawXXreverseDataset(args.validation_raw, args.validation_list, args.audio_window)
    train_loader = data.DataLoader(training_set, batch_size=args.batch_size, shuffle=True, **params) # set shuffle to True
    test_loader = data.DataLoader(test_set, batch_size=args.batch_size, shuffle=False, **params) # set shuffle to False
    # nanxin optimizer  
    optimizer = ScheduledOptim(
        optim.Adam(
            filter(lambda p: p.requires_grad, model.parameters()), 
            betas=(0.9, 0.98), eps=1e-09, weight_decay=1e-4, amsgrad=True),
        args.n_warmup_steps)

    model_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info('### Model summary below###\n {}\n'.format(str(model)))
    logger.info('===> Model total parameter: {}\n'.format(model_params))
    ## Start training
    best_acc = 0
    best_loss = np.inf
    best_epoch = -1 
    for epoch in range(1, args.epochs + 1):
        epoch_timer = timer()

        # Train and validate
        #trainXXreverse(args, model, device, train_loader, optimizer, epoch, args.batch_size)
        #val_acc, val_loss = validationXXreverse(args, model, device, validation_loader, args.batch_size)
        train(args, model, device, train_loader, optimizer, epoch, args.batch_size)
        val_acc, val_loss = validation(args, model, device, test_loader, args.batch_size)
        
        # Save
        if val_acc > best_acc: 
            best_acc = max(val_acc, best_acc)
            snapshot(args.logging_dir, run_name, {
                'epoch': epoch + 1,
                'validation_acc': val_acc, 
                'state_dict': model.state_dict(),
                'validation_loss': val_loss,
                'optimizer': optimizer.state_dict(),
            })
            best_epoch = epoch + 1
        elif epoch - best_epoch > 2:
            optimizer.increase_delta()
            best_epoch = epoch + 1
        
        end_epoch_timer = timer()
        logger.info("#### End epoch {}/{}, elapsed time: {}".format(epoch, args.epochs, end_epoch_timer - epoch_timer))
    
    ## end 
    end_global_timer = timer()
    logger.info("################## Success #########################")
    logger.info("Total elapsed time: %s" % (end_global_timer - global_timer))

# if __name__ == '__main__':
main()

===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval dataset
===> loading train, validation and eval 

RuntimeError: Expected hidden size (1, 10, 16), got [1, 2, 16]